
This capstone project has been submitted as part of the requirements for completion of the IBM Data Science Professional Certificate on Coursera. In general, this project would be encompassing a series of Data Science techniques, including, but not limited to, Web Scraping (using BeautifulSoup and Requests), Data Cleaning, Data Wrangling and Machine Learning (K-Means clustering algorithm).

Importing required libraries and packages


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 102kB 6.8MB/s 
Libraries imported.


Scraping web for neighborhood data of Munich (WikiPedia)

In [8]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Localities_of_Berlin").text
soup = BeautifulSoup(data, 'lxml')
textList = []
neighborhoodList = []

In [9]:

# append the data into the list
neighborhoodList.clear()

for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    
df = pd.DataFrame({"Neighborhood": neighborhoodList})
df1 = df.iloc[1:]
berlin_df = df1.reset_index(drop=True)
berlin_df.head()

,Neighborhood
0,Adlershof
1,Afrikanisches Viertel
2,Alt-Hohenschönhausen
3,Alt-Treptow
4,Altglienicke


In [10]:
# Geographical coordinates of neighborhoods

# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Berlin, Germany'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [ get_latlng(neighborhood) for neighborhood in berlin_df["Neighborhood"].tolist() ]

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
berlin_df['Latitude'] = df_coords['Latitude']
berlin_df['Longitude'] = df_coords['Longitude']


# check the neighborhoods and the coordinates
print(berlin_df.shape)
berlin_df.head()

(97, 3)


,Neighborhood,Latitude,Longitude
0,Adlershof,52.437790,13.54778
1,Afrikanisches Viertel,52.558269,13.33389
2,Alt-Hohenschönhausen,52.547060,13.50055
3,Alt-Treptow,52.493500,13.45711
4,Altglienicke,52.420060,13.53969


Visualise map with neighborhoods superimposed

In [12]:
# save the DataFrame as CSV file
berlin_df.to_csv("berlin_neighborhoods.csv", index=False)

In [13]:
address = 'Berlin, Germany'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Berlin is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Berlin is 52.5170365, 13.3888599.


In [14]:
# create map of Berlin using latitude and longitude values
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(berlin_df['Latitude'], berlin_df['Longitude'], berlin_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_berlin)  
    
map_berlin

Using Foursquare API

In [15]:
CLIENT_ID = 'MPE2IZ4NBGHBUAWJ0LON2S1TKCWGZQFT32KPBNVRE3PIZQTE'
CLIENT_SECRET = 'KSRMSHOJXPD04SGKZAY5HKXAUJKYHIRFA3ZYQLGVCP44JLHQ'
VERSION = '1622996925'

In [16]:
radius = 2000
LIMIT = 100

venues = []
for lat, long, neighborhood in zip(berlin_df['Latitude'], berlin_df['Longitude'], berlin_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [17]:

# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(5419, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adlershof,52.43779,13.54778,Mia Toscana,52.438327,13.549573,Italian Restaurant
1,Adlershof,52.43779,13.54778,McFIT,52.430956,13.549099,Gym / Fitness Center
2,Adlershof,52.43779,13.54778,Adapt Apartments Hotel,52.432655,13.532206,Hotel
3,Adlershof,52.43779,13.54778,dm-drogerie markt,52.437625,13.547692,Drugstore
4,Adlershof,52.43779,13.54778,Schloss Köpenick,52.443679,13.572549,Palace


In [18]:
venues_df.groupby(["Neighborhood"]).count()
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 341 uniques categories.


In [19]:

#### Analyse each neighborhood
# one hot encoding
onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

print(onehot.shape)

grouped = onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(grouped.shape)
grouped.head()

(5419, 342)
(97, 342)


,Neighborhoods,ATM,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Credit Union,Creperie,Cupcake Shop,Currywurst Joint,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Nudist Beach,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Paintball Field,Pakistani Restaurant,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pide Place,Pie Shop,Pier,Piercing Parlor,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Racecourse,Record Shop,Recreation Center,Rental Car Location,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Rugby Stadium,Russian Restaurant,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,

In [20]:
len(grouped[grouped["African Restaurant"] > 0])
berlin_rest = grouped[["Neighborhoods","African Restaurant"]]
berlin_rest

,Neighborhoods,African Restaurant
0,Adlershof,0.000000
1,Afrikanisches Viertel,0.011494
2,Alt-Hohenschönhausen,0.000000
3,Alt-Treptow,0.000000
4,Altglienicke,0.000000
5,Baumschulenweg,0.000000
6,Biesdorf (Berlin),0.000000
7,Blankenburg (Berlin),0.000000
8,Blankenfelde,0.000000
9,Bohnsdorf,0.000000


Clustering

In [21]:
kclusters = 4

berlin_clustering = berlin_rest.drop(["Neighborhoods"], 1)
berlin_clustering.head()

# run k-means clustering
kmeans = KMeans(init="k-means++", n_clusters=kclusters, n_init=12).fit(berlin_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [22]:
merged = berlin_rest.copy()

# add clustering labels
merged["Category"] = kmeans.labels_
merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
merged.head()

,Neighborhood,African Restaurant,Category
0,Adlershof,0.000000,0
1,Afrikanisches Viertel,0.011494,3
2,Alt-Hohenschönhausen,0.000000,0
3,Alt-Treptow,0.000000,0
4,Altglienicke,0.000000,0


In [23]:
dfmerged = merged.merge(berlin_df)
dfmerged.head()

#Sort
dfmerged.sort_values(["Category"], inplace=True, ascending=False)
dfmerged

,Neighborhood,African Restaurant,Category,Latitude,Longitude
1,Afrikanisches Viertel,0.011494,3,52.558269,13.33389
62,Neukölln (locality),0.020000,2,52.480770,13.43541
77,Schöneberg,0.010000,1,52.485550,13.34293
21,Friedenau,0.010000,1,52.472970,13.33269
89,Wedding (Berlin),0.010000,1,52.547810,13.35473
19,Fennpfuhl,0.010000,1,52.527730,13.46654
27,Gesundbrunnen (Berlin),0.010000,1,52.556190,13.37710
64,Niederschönhausen,0.000000,0,52.582650,13.40362
71,Reinickendorf (locality),0.000000,0,52.575450,13.34970
70,Rahnsdorf,0.000000,0,52.440930,13.68891


Visualising Clusters

In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfmerged['Latitude'], dfmerged['Longitude'], dfmerged['Neighborhood'], dfmerged['Category']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examining Clusters
Category 1: Neighborhoods with very low number of restaurants

In [25]:
merged.loc[merged['Category'] == 0]

,Neighborhood,African Restaurant,Category
0,Adlershof,0.0,0
2,Alt-Hohenschönhausen,0.0,0
3,Alt-Treptow,0.0,0
4,Altglienicke,0.0,0
5,Baumschulenweg,0.0,0
6,Biesdorf (Berlin),0.0,0
7,Blankenburg (Berlin),0.0,0
8,Blankenfelde,0.0,0
9,Bohnsdorf,0.0,0
10,Borsigwalde,0.0,0


Category 2: Neighborhoods with low number of restaurants

In [26]:
merged.loc[merged['Category'] == 1]

,Neighborhood,African Restaurant,Category
19,Fennpfuhl,0.01,1
21,Friedenau,0.01,1
27,Gesundbrunnen (Berlin),0.01,1
77,Schöneberg,0.01,1
89,Wedding (Berlin),0.01,1


Category 3: Neighborhoods with a significant number of restaurants

In [27]:
merged.loc[merged['Category'] == 2]

,Neighborhood,African Restaurant,Category
62,Neukölln (locality),0.02,2


Category 4: Neighborhoods crowded with restaurants

In [28]:
merged.loc[merged['Category'] == 3]

,Neighborhood,African Restaurant,Category
1,Afrikanisches Viertel,0.011494,3


Observations
It is clear the Category 3 are very crowded with African Restaurants, and hence, Category 1 would be the best bet for opening a new restuarant because of not too much competition in these regions, but still a proven market. Client with USPs to stand out from the competition can also open new restaurants in neighborhoods in Cluster 1 with moderate competition.